In [2]:
from pprint import pprint
import json
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [3]:
def apd_salary(i):
    
    mi, ma = '', ''
    result = ['','','']
    
    i = i.replace(u'\xa0', u'')
    i = i.replace(u'.', u'')
    
    if re.search('от ', i):
        min = re.findall('\d+', i)
        mi = min[0]
        
    if re.search('до ', i):
        max = re.findall('\d+', i)
        ma = max[0]
 
    if re.search('\d+-\d+', i):
        min = re.findall('\d+', i)
        max = re.findall('-\d+', i)
        max[0] = max[0].replace(u'-', u'')
        mi = min[0]
        ma = max[0]
        
    result[0] = mi
    result[1] = ma
    result[2] = i[-3:]
    
    return(result) 

In [4]:
def parsingSJ(html, site):
    res_block = html.find('div',{'class':'_3zucV'})
    res_list = res_block.findChildren(recursive=False)

    results=[]
    for vak in res_list:
        data = {}
        vak_salary = ['','','']
        try:    
            vak_name = vak.find('a',{'class':'bloko-link HH-LinkModifier'}).text#.findChild()
            vak_link = vak.find('a',{'class':'bloko-link HH-LinkModifier'})['href']
        except:
            continue           
    
        try:
            vak_s = vak.find('div',{'class':'vacancy-serp-item__compensation'}).getText()
            vak_salary = apd_salary(vak_s)
        except:
            pass

        data['name'] = vak_name
        data['salary_min'] = vak_salary[0]
        data['salary_max'] = vak_salary[1]
        data['salary_valuta'] = vak_salary[2]
        data['link'] = vak_link
        data['site'] = site

        results.append(data)    
        
    return(results)


main_link = 'https://www.superjob.ru'
#user_input = input()
link = main_link + '/vakansii/povar.html?geo%5Bt%5D%5B0%5D=4'#+user_input
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}
# &page=8
response = requests.get(link, headers=headers).text
html = bs(response,'lxml')

#pprint(response)
#html

res_block = html.findAll('span',{'class':'_3IDf-'})#[-1].getText()
max_page = 0
for i in res_block:
    try:
        if int(i.text):
            if max_page < int(i.text):
                max_page = int(i.text)
    except:
        pass

i = 1
results=[]

while i<max_page+1:
    print(int(100*i/(max_page+1)), ' %')
    results += parsingSJ(html, main_link)
    i += 1
    link1 = link + '&page=' + str(i)
    response = requests.get(link1, headers=headers).text
    html = bs(response,'lxml')

with open('data.json', 'w') as outfile:
    json.dump(results, outfile)

print('DONE!')

5  %
10  %
15  %
21  %
26  %
31  %
36  %
42  %
47  %
52  %
57  %
63  %
68  %
73  %
78  %
84  %
89  %
94  %
DONE!
